# RentHop: Rental Listing Inquiries

url = https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries/

### Understanding the Question

Given a set of features for a rental listing, we are to predict how much interest (low, medium, high) a rental listing will receive.  We are given labels for our data.  Our predictions should be represented as class probability (as per the competition rules).

This is a supervised classification problem.

### Getting Started - Load & Inspect Data

The data is available on kaggle at https://www.kaggle.com/c/two-sigma-connect-rental-listing-inquiries/data.  We are given 14 features in our data set and the label column is called 'interest_level'.

In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_json('train.json')
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street


In [2]:
train_df.shape

(49352, 15)

In [4]:
train_df.dtypes

bathrooms          float64
bedrooms             int64
building_id         object
created             object
description         object
display_address     object
features            object
interest_level      object
latitude           float64
listing_id           int64
longitude          float64
manager_id          object
photos              object
price                int64
street_address      object
dtype: object

In [5]:
#Check for NaNs
train_df.isnull().sum()

bathrooms          0
bedrooms           0
building_id        0
created            0
description        0
display_address    0
features           0
interest_level     0
latitude           0
listing_id         0
longitude          0
manager_id         0
photos             0
price              0
street_address     0
dtype: int64

### Feature Engineering

Before I start to visualized and look at descriptive statistics of the data, I want to convert some of the data fields into numerical features.

In [9]:
#Create # of Photos Column
train_df['NumPhotos'] = train_df.photos.str.len()
#Create # of Features Column
train_df['NumFeatures'] = train_df.features.str.len()

In [11]:
#Use sklearn LabelEncoder to label building_id and manager_id
from sklearn.preprocessing import LabelEncoder

#Encode building_id
le_building = LabelEncoder()
le_building.fit(train_df['building_id'])
train_df['BuildingID'] = le_building.transform(train_df['building_id'])

#Encode manager_id
le_manager = LabelEncoder()
le_manager.fit(train_df['manager_id'])
train_df['ManagerID'] = le_manager.transform(train_df['manager_id'])

#Inspect Changes
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address,NumPhotos,NumFeatures,BuildingID,ManagerID
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,5,0,2431,1239
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,11,5,5862,1583
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street,8,4,5806,2965
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street,3,2,1201,225
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street,3,1,0,2081


### Investigating Correlations

For my first attempt I want to make a rather simple model.  I will select only a handful of features that I think are relevant (based on my domain knowldge of the problem) and any that show a high correlation to the 'interest_level'.

In [17]:
#Select Features I want
feature_cols = ['bathrooms', 'bedrooms', 'price', 'latitude', 'longitude', 'NumPhotos', 'NumFeatures', 'BuildingID', 'ManagerID']
train_features = train_df[feature_cols]

#Describe for subsets of training based on 'interest_level'
train_features[train_df['interest_level'] == 'high'].describe()

,bathrooms,bedrooms,price,latitude,longitude,NumPhotos,NumFeatures,BuildingID,ManagerID
count,3839.000000,3839.000000,3839.000000,3839.000000,3839.000000,3839.000000,3839.000000,3839.000000,3839.000000
mean,1.116176,1.546496,2700.293045,40.748007,-73.964613,5.738474,5.158635,3622.783537,1741.772597
std,0.341725,1.112187,2080.554641,0.051965,0.040286,2.689384,4.167181,2280.917604,1054.697401
min,0.000000,0.000000,700.000000,40.575800,-74.159800,0.000000,0.000000,0.000000,11.000000
25%,1.000000,1.000000,1850.000000,40.721900,-73.990350,4.000000,2.000000,1618.000000,763.500000
50%,1.000000,2.000000,2400.000000,40.746500,-73.976300,5.000000,4.000000,3640.000000,1745.000000
75%,1.000000,2.000000,3163.000000,40.773800,-73.948600,7.000000,8.000000,5589.000000,2743.000000
max,4.000000,5.000000,111111.000000,41.086800,-73.714200,26.000000,32.000000,7578.000000,3480.000000


In [18]:
train_features[train_df['interest_level'] == 'medium'].describe()

,bathrooms,bedrooms,price,latitude,longitude,NumPhotos,NumFeatures,BuildingID,ManagerID
count,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000,11229.000000
mean,1.163906,1.622050,3158.767388,40.745567,-73.965033,5.813251,5.888681,3748.008816,1805.258527
std,0.388318,1.122604,1243.693856,0.388466,0.698923,2.654043,4.108768,2270.037277,1042.308607
min,0.000000,0.000000,695.000000,0.000000,-75.177300,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,2300.000000,40.726500,-73.991800,4.000000,3.000000,1708.000000,843.000000
50%,1.000000,2.000000,2895.000000,40.748800,-73.978100,5.000000,5.000000,3827.000000,1816.000000
75%,1.000000,2.000000,3650.000000,40.772400,-73.953300,7.000000,9.000000,5718.000000,2786.000000
max,4.000000,7.000000,15000.000000,44.603800,0.000000,28.000000,39.000000,7581.000000,3480.000000


In [19]:
train_features[train_df['interest_level'] == 'low'].describe()

,bathrooms,bedrooms,price,latitude,longitude,NumPhotos,NumFeatures,BuildingID,ManagerID
count,34284.000000,34284.000000,3.428400e+04,34284.000000,34284.000000,34284.000000,34284.000000,34284.000000,34284.000000
mean,1.238741,1.514759,4.176599e+03,40.739504,-73.951667,5.524647,5.307957,3006.765459,1781.091996
std,0.544946,1.111595,2.644932e+04,0.732933,1.355388,3.981574,3.820158,2482.302496,988.233997
min,0.000000,0.000000,4.300000e+01,0.000000,-118.271000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,2.625000e+03,40.729700,-73.991800,4.000000,2.000000,328.000000,933.000000
50%,1.000000,1.000000,3.300000e+03,40.753800,-73.977900,5.000000,4.000000,2926.500000,1788.000000
75%,1.000000,2.000000,4.400000e+03,40.774725,-73.956000,7.000000,8.000000,5150.000000,2679.000000
max,10.000000,8.000000,4.490000e+06,44.883500,0.000000,68.000000,31.000000,7584.000000,3480.000000


Just a quick look at these basic descriptive statistics and it seems 'price' is very influential in the interest level.  The high interest subset has a much lower mean and median price than the rest of the data.

### Prepare Data for ML & Transform Test Features

With this smaller set of features, I want to scale it and then run it through a few classification algorithims to see what intial results I can get from a simple model. 

In order to do that I will first need to apply the same transforms to the test features.  Then I will scale the training and test features so they can be better utilized by a machine learning algorithm.

#### Apply same transforms to test features

In [23]:
#Load test data
test_df = pd.read_json('test.json')
#Create # of Photos Column
test_df['NumPhotos'] = test_df.photos.str.len()
#Create # of Features Column
test_df['NumFeatures'] = test_df.features.str.len()
#Encode manager_id -already trained, just fit
test_df['ManagerID'] = le_manager.transform(test_df['manager_id'])
#Encode building_id -already trained, just fit
test_df['BuildingID'] = le_building.transform(test_df['building_id'])

#Inspect to verify
test_df.head()

ValueError: y contains new labels: [u'0115c7abb8f1d3b50673f3af09fffd0e' u'011f9959a21777007f3cfed4d9138d15'
 u'01a202519a4a0e384ac97a1bf5f435b9' u'01b8e7749ec880170b33dd6a01216f14'
 u'01dab36b00b81746903c9577164f223a' u'02083c95193790b1acf5428e55d53947'
 u'0220efae47fbe40afbfbac95bbc7f239' u'0270de3c58a9285cef164e611aa0e79b'
 u'0279ea9e992f8108e39c9d2318dc2b98' u'028564cb4b3172a81589960afa143aac'
 u'032d78195212687c678990b2a7785b9d' u'0347071240c5d5eb0ed3c438e8f41a18'
 u'03a56d25f4fa1845d84325013b8e1cd2' u'03bb0ee3e78614a9dca4229451adfd71'
 u'04b3769d9e4676d9c22dbf8df4a63541' u'04c332720bb43160801867c75b6ba345'
 u'05604c02fd5fdf849464fa94e974faf0' u'056aea3f29ceb159af1d8edb5b4f3564'
 u'057a1cdfa6bf51d55a5baf9109521bfb' u'057e0cb0cec05f29190c9985614546f5'
 u'05959ca788e96cab3a88a7650a8cd183' u'05d06180ba2b7e04f411c4889488861a'
 u'0693b188538f4c0d32d40d49493eee24' u'07329b910729d73d5ba945f0e03e77c0'
 u'0764f425b4186cae92225bb562541456' u'078e957430b1ee48a4ba8c49b1709f4f'
 u'0839b83ec3989

#### Fixing this Encoding Error

This error is basically saying that in the test set there are new buildings/ managers that were not in the training set and therefore not encoded.  To fix this we need to combine these columns from the train and test set and fit our encoder to that set.  Then apply the transform to both the train and test set.

In [29]:
#Combine the train and test columns
manager_combo = train_df['manager_id'].append(test_df['manager_id'])
building_combo = train_df['building_id'].append(test_df['building_id'])

#Encode building_id
le_building = LabelEncoder()
le_building.fit(building_combo)
#Transform Train & Test set
train_df['BuildingID'] = le_building.transform(train_df['building_id'])
test_df['BuildingID'] = le_building.transform(test_df['building_id'])

#Encode manager_id
le_manager = LabelEncoder()
le_manager.fit(manager_combo)
#Transform Train & Test set
train_df['ManagerID'] = le_manager.transform(train_df['manager_id'])
test_df['ManagerID'] = le_manager.transform(test_df['manager_id'])

#Inspect to verify
train_df.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,interest_level,latitude,listing_id,longitude,manager_id,photos,price,street_address,NumPhotos,NumFeatures,BuildingID,ManagerID,IL
10,1.5,3,53a5b119ba8f7b61d4e010512e0dfc85,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,[],medium,40.7145,7211212,-73.9425,5ba989232d0489da1b5f2c45f6688adc,[https://photos.renthop.com/2/7211212_1ed4542e...,3000,792 Metropolitan Avenue,5,0,3797,1568,2
10000,1.0,2,c5c8a357cba207596b04d1afd1e4f130,2016-06-12 12:19:27,,Columbus Avenue,"[Doorman, Elevator, Fitness Center, Cats Allow...",low,40.7947,7150865,-73.9667,7533621a882f71e25173b27e3139d83d,[https://photos.renthop.com/2/7150865_be3306c5...,5465,808 Columbus Avenue,11,5,8986,1988,1
100004,1.0,1,c3ba40552e2120b0acfc3cb5730bb2aa,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,"[Laundry In Building, Dishwasher, Hardwood Flo...",high,40.7388,6887163,-74.0018,d9039c43983f6e564b1482b273bd7b01,[https://photos.renthop.com/2/6887163_de85c427...,2850,241 W 13 Street,8,4,8889,3733,0
100007,1.0,1,28d9ad350afeaab8027513a3e52ac8d5,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,"[Hardwood Floors, No Fee]",low,40.7539,6888711,-73.9677,1067e078446a7897d2da493d2f741316,[https://photos.renthop.com/2/6888711_6e660cee...,3275,333 East 49th Street,3,2,1848,282,1
100013,1.0,4,0,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,[Pre-War],low,40.8241,6934781,-73.9493,98e13ad4b495b9613cef886d79a6291f,[https://photos.renthop.com/2/6934781_1fa4b41a...,3350,500 West 143rd Street,3,1,0,2618,1


In [30]:
from sklearn.preprocessing import StandardScaler

#Scale & Set Feature Values
X_train = StandardScaler().fit_transform(train_features.values)
X_test = StandardScaler().fit_transform(test_df[feature_cols].values)


#Encode 'interest_level' to numerical
le_interest = LabelEncoder()
train_df['IL'] = le_interest.fit_transform(train_df['interest_level'])
#Set Train Y
Y = train_df['IL'].values
#Inspect to verify
Y [:10]

array([2, 1, 0, 1, 1, 2, 1, 1, 2, 1], dtype=int64)

In [46]:
#Get Label encodings for reference later
le_interest.classes_

array([u'high', u'low', u'medium'], dtype=object)

### Test a few ML Algorithims

Now the data is prepared to test a few different ML approaches.  Using cross-validation on a smaller set of the training data I will test a few different models.  Since the competition is being evaluated by using log loss, we will set the cross validation scorer in sklearn to use that method as well.

#### Naive Bayes

In [32]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.naive_bayes import GaussianNB

#Initialize Model
nb_model = GaussianNB()
#Create KFold
kfold = KFold(n_splits=7, random_state=5)
cross_val_results = cross_val_score(nb_model, X_train, Y, cv=kfold, scoring='neg_log_loss')
print cross_val_results.mean()

-4.57676082504


#### Logistic Regression

In [33]:
from sklearn.linear_model import LogisticRegression

#Initialize Model
log_model = LogisticRegression()
#Create KFold
kfold = KFold(n_splits=7, random_state=5)
cross_val_results = cross_val_score(log_model, X_train, Y, cv=kfold, scoring='neg_log_loss')
print cross_val_results.mean()

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\linear_model\base.py:352: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


-0.71154068797


### Train Model & Make First Submission


In [36]:
#Logistic Regression
log_model = LogisticRegression()
log_model.fit(X_train, Y)
prediction_probabilites = log_model.predict_proba(X_test)
prediction_probabilites[:10, :]

array([[  1.54128357e-01,   4.96697636e-01,   3.49174007e-01],
       [  1.74563954e-01,   5.10360311e-01,   3.15075735e-01],
       [  2.61561754e-02,   8.60331585e-01,   1.13512239e-01],
       [  1.54292599e-01,   4.59562735e-01,   3.86144666e-01],
       [  8.69627589e-03,   8.72590474e-01,   1.18713251e-01],
       [  5.54461164e-06,   9.98125304e-01,   1.86915136e-03],
       [  2.52257363e-01,   3.57603482e-01,   3.90139155e-01],
       [  3.59407483e-01,   2.81115697e-01,   3.59476820e-01],
       [  1.49331328e-01,   5.41461157e-01,   3.09207515e-01],
       [  2.55397888e-01,   4.20177473e-01,   3.24424639e-01]])

In [55]:
#Submission must be - listing_id, high, medium, low
#The index of our probabilties is from the label encoder earlier (0=high, 1=low, medium=2)
submission_df = pd.DataFrame({'listing_id':test_df['listing_id'], 'high':prediction_probabilites[:, 0],
                             'medium':prediction_probabilites[:, 2], 'low':prediction_probabilites[:, 1]})
#Re-Order Columns for submission
cols = ['listing_id', 'high', 'medium', 'low']
submission_df = submission_df[cols]
submission_df.head()

,listing_id,high,medium,low
0,7142618,0.154128,0.349174,0.496698
1,7210040,0.174564,0.315076,0.510360
100,7103890,0.026156,0.113512,0.860332
1000,7143442,0.154293,0.386145,0.459563
100000,6860601,0.008696,0.118713,0.872590


In [50]:
#Verify all is well (no NaNs)
submission_df.isnull().sum()

high          0
listing_id    0
low           0
medium        0
dtype: int64

In [54]:
#Write to CSV for submission
submission_df.to_csv('log_reg.csv', index=False)

### Conclusion

This simple Logistic Regression scored .8516 on Kaggle.  Not very good, a lot of room for improvement.  The first thing to do is tune the hyper-parameters for our logistic regression.